<a href="https://colab.research.google.com/github/zanattopaolo1/OOP-git-merge-conflict-test/blob/master/ML_project_Zanatto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This is (....)

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
#import math
import pandas as pd
from tabulate import tabulate
import csv

# Functions
List of functions that will be used inside the code

In [ ]:
def duplicate_check(df, cols):
    # Check for duplicates based on the specified columns
    duplicate_rows = df[df.duplicated(subset=cols, keep=False)]

    return duplicate_rows

"""def filling_UltimoPrelievo(df):
    #Having GiornoDaTerapiaPrecedente equal to GiorniTeoriciDaTerapiaPrecedente means that the schedule has been respected. Considering that the cases with we are assuming
    df['UltimoPrelievo'] = df['UltimoPrelievo'].where(df['UltimoPrelievo'] == '' & df['GiornoDaTerapiaPrecedente']==df['GiorniTeoriciDaTerapiaPrecedente'], df['DataSomministrazione'] - pd.DateOffset(days=1))

    #Removing data where GiornoDaTerapiaPrecedente is NOT equal to GiorniTeoriciDaTerapiaPrecedente
    (...)"""


def failed_blood_test(df):
    condition1 = df['GiornoDaTerapiaPrecedente'] > df['GiorniTeoriciDaTerapiaPrecedente']  # Define your condition here
    condition2 = df['NumeroPrelievi'] > 1
    filtered_df = df[condition1 | condition2]
    return filtered_df;

def del_rows_missing_val (df):
    condition = df['UltimoPrelievo'] == ''

    rows_to_drop = df[condition].index
    return df.drop(rows_to_drop)

def data_cleaning(df):
    #conditions to drop rows
    condition1 = df['UltimoPrelievo'] < df['PrimoPrelievo'] #this information would be caused by a collection error
    #condition2 = True   & condition2

    rows_to_drop = df[condition1].index
    return df.drop(rows_to_drop)

"""def strange_blood_test(df):
    condition = df['GiornoDaTerapiaPrecedente'] < df['GiorniTeoriciDaTerapiaPrecedente'] # Define your condition here
    filtered_df = df[condition]
    return filtered_df;"""

def attrib_removal(df, attrib_list):
    for x in attrib_list:
        df = df.drop(x, axis=1)

    return df

def check_value_presence(row):
    return {f'{value}': value in row['PrincipiAttivi'] for value in unique_values_set}

# Data loading

Data import from the csv file into a dictionary, to structure the information in a logical way, and then used to create the Dataframe

In [ ]:
dataset_file_path = 'dataset_farmacia_anonimo(3).csv'

keys = ['IDPaziente',	'genere',	'classeeta',	'IDTerapia',	'CodiceSedeTumore',	'PrincipiAttivi',	'PrincipiAttiviDescrizione',	'NumeroSomministrazione',	'DataSomministrazioneProgrammata',	'DataSomministrazione',	'DataSomministrazioneProgrammataPrecedente',	'DataSomministrazionePrecedente',	'GiornoDaTerapiaPrecedente',	'GiorniTeoriciDaTerapiaPrecedente',	'PrimoPrelievo',	'UltimoPrelievo', 'NumeroPrelievi']
b_tests = {x: [] for x in keys}

with open(dataset_file_path, 'r') as file:
    reader = csv.reader(file, delimiter=";")
    for row_index, row in enumerate(reader):
        if row_index > 0:
            for idx, cell in enumerate(row):
                if "," in cell:             #to save the list of active ingredients as array
                    b_tests[keys[idx]].append(cell.split(","))
                else:
                    b_tests[keys[idx]].append(cell)

df = pd.DataFrame(b_tests)

Data visualization

In [ ]:
print(df.to_string(index=False))

#print(tabulate(df, df.columns.tolist(), tablefmt='grid'))

# Data pre processing
The data presents many issues that requires to be fixed

1. Duplicated data: Having duplicated data would mean considering multiple times the same information and it will bring a distorsion to reality, so a check is relavant

In [ ]:
dup_rows = duplicate_check(df, ['IDPaziente', 'IDTerapia', 'NumeroSomministrazione'])
print("\n\nDuplicated rows:\n", dup_rows)

2.  Missing values: There are many missing values in the fields "PrimoPrelievo" and "UltimoPrelievo" and I considered the option to fill "UltimoPrelievo". The plan was to fill that field with "DataSomministrazione"-1 for each case where the "GiornoDaTerapiaPrecedente" is equal to "GiorniTeoriciDaTerapiaPrecedente" (so the schedule has been respected) but this option was assuming that all these therapy executions were scheduled on the last 4 days of the week. This is not a feasible assumption when you are interested to compare the results in percentage of blood tests done 1 day beforeand the ones done 3 days before, as this project aims to do. So below the rows where the "UltimoPrelievo" (and jointly "PrimoPrelievo") are missing will be deleted.

In [ ]:
df = del_rows_missing_val(df)

3.  Casting of data: the import functions extract the data as strings. It makes a problem when the data has to be computed for what they really are.

In [ ]:
df['NumeroPrelievi'] = df['NumeroPrelievi'].astype(int)
df['DataSomministrazione'] = pd.to_datetime(df['DataSomministrazione'], format='%d/%m/%Y')
df['UltimoPrelievo'] = pd.to_datetime(df['UltimoPrelievo'], format='%d/%m/%Y')
df['PrimoPrelievo'] = pd.to_datetime(df['PrimoPrelievo'], format='%d/%m/%Y')

4.  Noise: There may occur some error in collection phase of data, so it is important to verify the correctness by means of conditions based on logic of data. These are reported and used in the function "data_cleaning".

In [ ]:
df = data_cleaning(df)

5.  Creation of attribute for the class

The goal of this phase is to create a classifier that allow to reach the goal of the algorithm.

In order to make it, the first step is to define the classes.
In this problem the classes are:


*   Ready for the therapy
*   Not ready for the therapy

To distringuish them it has been used 2 conditions:


*   Day of previous therapy is bigger than the scheduled day of previous therapy
*   The number of blood test is bigger than 1

The rows that reflect at least one of these 2 conditions are considered as "not ready for the therapy". Theoretically both of conditions should be valid in the same moment and using only one of them would be sufficient, but in this way it helps to consider the undetected noise of the dataset.



In [ ]:
failed_btests = failed_blood_test(df)

print("\n\nThere are {} failed blood tests: {}\n".format(len(failed_btests), tabulate(failed_btests, df.columns.tolist(), tablefmt='grid')))
print("\n\nThere are {} failed blood tests ({} % of the total)\n".format(len(failed_btests), len(failed_btests)/len(df)*100))

There is a strong incidence of failed blood tests and it is important to keep in mind it in front of the solution.

Given the rows belonging to the classes, it is created a new column to assign the class.

In [ ]:
new_column_values = np.where(df.isin(failed_btests).all(axis=1), False, True)
df['Single_btest'] = new_column_values

In [ ]:
#To confirm that the creation has done correctly:
#print(tabulate(df, df.columns.tolist(), tablefmt='grid'))
#print("\nPercentage of failures of the first blood test: {} %".format((df['Single_btest']==False).sum() / len(df) * 100))

For the purpose of the project has been fundamental to create also another column to count the day passed between blood test and therapy execution.  

In [ ]:
iff_days = (df['DataSomministrazione'] - df['UltimoPrelievo']).dt.total_seconds().fillna(-1).astype(int)
df['Dist_bt_th'] = round(diff_days / (3600 * 24))

print(tabulate(df, df.columns.tolist(), tablefmt='grid'))
print("\nPercentage of blood tests executed with more than 1 days before therapy execution: {} %".format((df['Dist_bt_th']>1).sum() / len(df) * 100))
print("\nPercentage of blood tests executed 3 days before therapy execution: {} %".format((df['Dist_bt_th']==3).sum() / len(df) * 100))


6.  Removal of useless attributes: there are some attributes that are not relevant to be kept for the working operations of the ML algorithm:



In [ ]:
list = ['IDPaziente', 'IDTerapia', 'PrincipiAttiviDescrizione', 'NumeroSomministrazione',
        'DataSomministrazioneProgrammata', 'DataSomministrazione', 'DataSomministrazioneProgrammataPrecedente',
        'DataSomministrazionePrecedente', 'GiornoDaTerapiaPrecedente',	'GiorniTeoriciDaTerapiaPrecedente',
        'PrimoPrelievo', 'UltimoPrelievo', 'NumeroPrelievi']
df = attrib_removal(df, list)
print(df.columns.tolist())

7.  Management of the attribute "PrincipiAttivi" that contains array: this attribute contains an array of active ingredients, but in this format it does not suit with the algorithm operations.


> So, to make it usable, it can be created an attribute for each possible value and then setting as True for the rows where that value is present.



In [ ]:
df = pd.concat([df, df.apply(check_value_presence, axis=1, result_type='expand')], axis=1)

# Model generation

This section will be focused on the concrete creation of the classifier.

The first step is the split of the dataset with its classes and, after that, they are divided into:

*   Training set (70%)
*   Test set (20%)
*   Validation set (10%)

In [ ]:
th_classes = df[['Single_btest', 'Dist_bt_th']]
df = df.drop(['Single_btest', 'Dist_bt_th'], axis=1)

cutpoint1, cutpoint2 = 0.7, 0,8
trainset = df[:int(cutpoint1*len(df))]
train_cl = th_classes[:int(cutpoint1*len(df))]
valset = df[int(cutpoint1*len(df)):int(cutpoint2*len(df))]
val_cl = th_classes[int(cutpoint1*len(df)):int(cutpoint2*len(df))]
testset = df[int(cutpoint2*len(df)):]
test_cl = th_classes[int(cutpoint2*len(df)):]
print('Total: {} splitted in Train: {}, Val: {} and Test: {}'.format(len(df), len(trainset), len(valset), len(testset)))

Now it's the turn for the creation of the classifier.

There are many possibilities and here we are goinf to try the Support Vector Machines. The parameters used are the following ones (considering this [source](https://vitalflux.com/svm-rbf-kernel-parameters-code-sample/#:~:text=The%20gamma%20parameter%20defines%20how,high%20values%20meaning%20'close'.)):


*   kernel = rbf: the dimensionality of data is low
*   C = 10: it aims to reduce regularization
*   gamma = 0.05: to guarantee sufficient influence of training set



In [ ]:
clf = svm.SVC(gamma=0.1, C=10, kernel='rbf')